## Data Extraction and Text Analysis Blackcoffer Consulting


### Data Extraction


For each of the articles, given in the input.xlsx file, extract the article text and save the extracted article in a text file with URL_ID as its file name.


first extract the required libraries

In [1]:
import re
import nltk
import string
import urllib
import numpy as np
import pandas as pd
import requests as rs
from bs4 import BeautifulSoup as soup


In [2]:
#pip install textblob

In [3]:
from textblob import TextBlob

#### using vaderSentiment package to count positive and negative score

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


### Sampling of data extraction

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}
url = 'https://insights.blackcoffer.com/how-small-business-can-survive-the-coronavirus-crisis/'
page = rs.get(url,headers=headers)

In [6]:
a = soup(page.text, "html.parser")


In [7]:
title = a.find('title').get_text()
print(str(title))
para = [x.get_text() for x in a.find_all('p')]
print(type(para))

print(title+str(para))


How Small Business can survive the Coronavirus Crisis - Blackcoffer Insights
<class 'list'>
How Small Business can survive the Coronavirus Crisis - Blackcoffer Insights['The word pandemic may be the most used word for the last decade, and we know why. The year 2020 comes with a lot of unprecedented threats to humans as well as to nature. While no business was left insulated from its effect, small businesses were one of the most affected, as they rely on day to day sales and keep the inventory minimum, based on the demand of customers and have a low margin to maneuver.', 'In an article by HSBC’s Navigator, which described the preparedness and contingency plan for the future, Indian business rated high among its peers. The businesses in India have shown resilience to face adversity which can be attributed to varying degrees of adjustment and adaptability. The businesses, while able to adhere to government guidelines have managed to facilitate their customers efficiently and effectively. 

extract the input file and read the content of the file

In [8]:
data= pd.read_excel('Input.xlsx')
print(data.shape)

(170, 27)


### storing the content in text file with URL as a file name

sice files can not be stored in url format some changes are made like reolacing '/' with '$' and ':' with '#'

In [9]:
def encode_url(url):
    return url.replace("/", "$").replace(":", "#") 

creating a function to copy the ectracted data to a text file

In [10]:
def content_urls(urls):   
    page = rs.get(urls,headers=headers)
    a = soup(page.text, "html.parser")
    title = a.find('title').get_text()
    para = [x.get_text() for x in a.find_all('p')]
    file_name= encode_url(urls)
    file_name = str(file_name)
    #file_name.replace("/", "_").replace(":", "#")
    print(str(file_name))
    file = open('%s.txt' % file_name,'w',encoding="utf8")
    file.write(str(title))
    file.write(str(para))
    #file.flush()
    file.close()
    return 
    

calling the above function

In [11]:
#data['URL'].apply(lambda x: content_urls(x))

### created a function called content_urls and passes each url to it and storing the content and title to a column called content rather than creating text file

In [12]:
def content_urls_data(urls):   
    page = rs.get(urls,headers=headers)
    a = soup(page.text, "html.parser")
    title = a.find('title').get_text()
    para = [x.get_text() for x in a.find_all('p')]
    para = str(para)
    content = title+para
    return content

In [13]:
#data['content'] = data['URL'].apply(lambda x: content_urls_data(x))

In [14]:
data.to_csv('Input.csv',index=False)

In [15]:
data1 = pd.read_csv("Input.csv")

In [16]:
df = data1

In [17]:
df.dropna(axis = 1, inplace = True)

In [18]:
print(df.content.dtype)
    

object


## DATA ANALYSIS

For each of the extracted texts from the article, perform textual analysis and compute variables


#### Function is used to remove the special character

Here I have removed the special character only keeping '.'(fullstop) so to distinguish the sentence

In [19]:
def clean_data(input_text):
    input_text = str(input_text)
    input_txt= re.sub("[^A-Za-z0-9.]+", " ",input_text)
    return input_txt

In [20]:
df['cleaned_data'] = df['content'].apply(lambda x: clean_data(x))

In [21]:
df.cleaned_data[0] # sample data set

' title How is Login Logout Time Tracking for Employees in Office done by AI Blackcoffer Insights title p When people hear AI they often think about sentient robots and magic boxes. AI today is much more mundane and simple but that doesn t mean it s not powerful. Another misconception is that high profile research projects can be applied directly to any business situation. AI done right can create an extreme return on investments ROIs for instance through automation or precise prediction. But it does take thought time and proper implementation. We have seen that success and value generated by AI projects are increased when there is a grounded understanding and expectation of what the technology can deliver from the C suite down. p p Artificial Intelligence AI is a science and a set of computational technologies that are inspired by but typically operate quite differently from the ways people use their nervous systems and bodies to sense learn reason and take action. 3 Lately there has 

#### remove the wordes whose length is less than 3

In [22]:
#df['cleaned_data'] = df['cleaned_data'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [23]:
#df['cleaned_data'] = df['cleaned_data'].apply(lambda x: x.split('.'))

In [24]:
df['cleaned_data'].shape

(170,)

### POSITIVE SCORE calculation

In [25]:
def pos_scores(statement):
    sid_obj = SentimentIntensityAnalyzer()
    scores  = sid_obj.polarity_scores(str(statement))
    return scores['pos']
    

In [26]:
df['POSITIVE SCORE']= df['cleaned_data'].apply(lambda x: pos_scores(x))

### NEGATIVE SCORE calculation

In [27]:
def negative_scores(statement):
    sid_obj = SentimentIntensityAnalyzer()
    scores  = sid_obj.polarity_scores(str(statement))
    return scores['neg']

In [28]:
df['NEGATIVE SCORE']= df['cleaned_data'].apply(lambda x: negative_scores(x))

### POLARITY SCORE calculation

Polarity tells whether thegiven text is a positive text or negative the value ranges fron[-1,1]

In [29]:
def polarity_score(statement):
    statement = str(statement)
    polarity = TextBlob(statement).sentiment
    return polarity[0]

In [30]:
df['POLARITY SCORE'] = df['cleaned_data'].apply(lambda x: polarity_score(x))

### SUBJECTIVITY SCORE calculation

Subjective sentences generally refer to personal opinion, emotion or judgment . So the below code describes the if the provided text is subjective or objective

In [31]:
def sunjective_score(statement):
    statement = str(statement)
    subjectivity = TextBlob(statement).sentiment
    #print(subjectivity)
    return subjectivity[1]

In [32]:
df['SUBJECTIVITY SCORE'] = df['cleaned_data'].apply(lambda x: sunjective_score(x))

In [33]:
TextBlob(str(df['cleaned_data'][1])).sentiment

Sentiment(polarity=0.1746273660979544, subjectivity=0.48629912571089035)

In [34]:
#pip install vaderSentiment

In [35]:
sid_obj = SentimentIntensityAnalyzer()
sid_obj.polarity_scores(str(df['cleaned_data'][0]))

{'neg': 0.042, 'neu': 0.887, 'pos': 0.071, 'compound': 0.9709}

### AVG SENTENCE LENGTH calculation

In [36]:
def avg_sentence_len(text):
    sentences = text.split(".") #split the text into a list of sentences.
    words = text.split(" ") #split the input text into a list of separate words
    if(sentences[len(sentences)-1]==""): #if the last value in sentences is an empty string
        average_sentence_length = len(words) / len(sentences)-1
    else:
        average_sentence_length = len(words) / len(sentences)
    return average_sentence_length 

In [37]:
df['AVG SENTENCE LENGTH'] = df['cleaned_data'].apply(lambda x: avg_sentence_len(x))

### WORD COUNT calculation

In [38]:
def wordcount(text):
    # Regex to match all words, hyphenated words count as a compound words
    return len(re.findall("[a-zA-Z-]+", text))

In [39]:
df['WORD COUNT'] = df['cleaned_data'].apply(lambda x: wordcount(x))

### AVG WORD LENGTH calculation

In [40]:
def average_word_length(sentence):
    words = sentence.split()
    average = sum(len(word) for word in words) / len(words)
    return average

In [41]:
df['AVG WORD LENGTH'] = df['cleaned_data'].apply(lambda x: average_word_length(x))

### SYLLABLE PER WORD calculation

In [42]:
def syllable_count(text):
    syllable_count=0
    for word in text:
        for w in word:
            if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'or w == 'y'or w == 'Y'):
                syllable_count=syllable_count+1
    return syllable_count

In [43]:
df['SYLLABLE PER WORD'] = df['cleaned_data'].apply(lambda x: syllable_count(x))

### PERSONAL PRONOUNS calculation

In [44]:
def personal_pronoune(text):
    syllable_count=0
    for word in text:
        for w in word:
            if(w=='he' or w=='she' or w=='i' or w=='He' or w=='She' or w=='I' or w=='us' or w=='Us'):
                syllable_count=syllable_count+1
    return syllable_count

In [45]:
df['PERSONAL PRONOUNS'] = df['cleaned_data'].apply(lambda x: personal_pronoune(x))

### COMPLEX WORD COUNT calculation

In [46]:
def complex_word_count(statement):
    statement = str(statement)
    words = ' '.join([w for w in statement.split() if len(w)>3])
    #print(words)
    x = len(re.findall("[a-zA-Z-]+", words))
    #print(x)
    return x
    

In [47]:
df['COMPLEX WORD COUNT']=df['cleaned_data'].apply(lambda x: complex_word_count(x))

### PERCENTAGE OF COMPLEX WORDS calcualtion

In [48]:
df['PERCENTAGE OF COMPLEX WORDS'] = (df['COMPLEX WORD COUNT']/df['WORD COUNT'])*100
    

 ### FOG INDEX calculation

In [49]:
df['FOG INDEX'] = (df['AVG SENTENCE LENGTH']+ df['PERCENTAGE OF COMPLEX WORDS'])* 0.4

In [50]:
def avrg_word_per_sentense(text):
    words = text.split()
    wordCount = len(words)
    for word in words:
        ch= len(word)
    avg = ch / wordCount
    return avg

In [51]:
df['AVG NUMBER OF WORDS PER SENTENCE']= df['cleaned_data'].apply(lambda x: avrg_word_per_sentense(x))

In [52]:
dff = df

In [53]:
dff.drop('content', axis =1,inplace = True)

In [54]:
dff.drop('cleaned_data', axis = 1, inplace = True)

In [57]:
dff.to_csv('Output.csv', index = False)

In [56]:
dff

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,WORD COUNT,AVG WORD LENGTH,SYLLABLE PER WORD,PERSONAL PRONOUNS,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE
0,1,https://insights.blackcoffer.com/how-is-login-...,0.071,0.042,0.136284,0.462936,26.821429,745,4.878505,1480,266,474,63.624161,36.178236,0.001335
1,2,https://insights.blackcoffer.com/how-does-ai-h...,0.142,0.036,0.174627,0.486299,22.793103,660,5.177542,1342,264,428,64.848485,35.056635,0.001517
2,3,https://insights.blackcoffer.com/ai-and-its-im...,0.168,0.023,0.182008,0.565414,25.233766,1939,5.028336,3832,819,1178,60.752965,34.394693,0.000515
3,4,https://insights.blackcoffer.com/how-do-deep-l...,0.094,0.000,0.091798,0.462051,30.066667,449,5.280624,955,201,297,66.146993,38.485464,0.002227
4,5,https://insights.blackcoffer.com/how-artificia...,0.219,0.021,0.209313,0.609319,24.812500,787,4.940657,1537,340,474,60.228717,34.016487,0.001263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,0.088,0.029,0.133209,0.430751,18.560440,1694,5.062833,3319,596,1039,61.334120,31.957824,0.000593
166,168,https://insights.blackcoffer.com/sales-forecas...,0.125,0.054,0.095859,0.469022,23.645161,731,5.502052,1564,337,498,68.125855,36.708407,0.001368
167,169,https://insights.blackcoffer.com/detect-data-e...,0.120,0.104,0.066126,0.503678,15.333333,1134,5.284843,2313,478,757,66.754850,32.835273,0.000871
168,170,https://insights.blackcoffer.com/data-exfiltra...,0.051,0.032,0.062619,0.505134,13.934783,643,4.928013,1148,237,395,61.430793,30.146230,0.001565
